In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import datetime
import dask.array

import sys
sys.path.append('../')
import snowFun

In [2]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_mask = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks')

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi, drop='geometry')

In [3]:
### choose if you want to do only the 45 validation glaciers
validation_only = 0

# select which rgis to analyze
if validation_only:
    folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Validation')
    
else:
    folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')

# get list of rgis to analyze
# rgis_to_analyze = list(set( [ i[3:17] for i in os.listdir(os.path.join(folder_save, "Average AAs")) if i[-3:]=='tif' ] ))

# # get list of glacier area for each rgi
# areas = [rgi_gdf[rgi_gdf['RGIId']==i]['Area'].values for i in rgis_to_analyze]

# # make df
# rgis_to_analyze_df = pd.DataFrame({"RGIId":rgis_to_analyze, 'Area':areas})

# # sort however you want
# rgis_to_analyze_df = rgis_to_analyze_df.sort_values('Area')

# # grab rgi names
# rgis_to_analyze = rgis_to_analyze_df['RGIId'].values

# print(len(rgis_to_analyze_df))
# # print(rgis_to_analyze[:10])
# # print(rgis_to_analyze_df[:10])

### First step: open glacier DEM and calculate hypsometric ELA given observed AAR, hypsometric AAR given ELA
#### Then calculate the AAR if we raised ELA 10m, 20m, 30m, etc...

In [4]:
# open final aar/ela df
df_path = os.path.join(folder_save, 'Average AAs', 'final_glacier_stats.csv')
df_obs = pd.read_csv(df_path)

# get rgi list
rgis_to_analyze = df_obs['RGIId'].values

# initiate list to hold outputs
outputs = []

for i in range(len(rgis_to_analyze)):
    
    # subset rgi to single outline, by choosing rgiid or rgi name
    rgiid = rgis_to_analyze[i]
    
#     if rgiid!="RGI60-01.09162": continue

    # quickly grab glacier area
    ga = rgi_gdf[rgi_gdf['RGIId']==rgiid]['Area'].values[0]

    # print progress
    print(f"Starting {i+1} of {len(rgis_to_analyze)}: {rgiid}  {ga} km2")
    
    # grab just this rgi geometry and info
    rgi_single = rgi_gdf[rgi_gdf['RGIId']==rgiid].to_crs("EPSG:3338")
    single_geometry = rgi_single.geometry
    single_obs = df_obs[df_obs['RGIId']==rgiid]
    
    # define the coarsen scale
#     if ga>3000:
#         scale=5
    if ga>1000:
        scale=5
    elif ga>500:
        scale=3
    else:
        scale=1
        
    # open glacier mask
    glacier_mask = xr.open_dataset(os.path.join(folder_mask, f"S2_{rgiid}_mask.nc"), chunks='auto').glacier
    
#     if ga>500:
#         glacier_mask = glacier_mask.coarsen({"x":scale, "y":scale}, boundary="trim").median(skipna=True).astype('uint8')  

    # open dem for year 2020
    xr_dem = snowFun.get_year_DEM(single_geometry, 2020, smoothed=0) 
    
    # coarsen dem
#     if ga>500:
#         xr_dem = xr_dem.sel({"x":glacier_mask.x, "y":glacier_mask.y})

    # shave off edges to make sure dem, mask match
    xr_dem = xr_dem.sel(x=slice( min(glacier_mask.x.values), max(glacier_mask.x.values) ), y=slice(max(glacier_mask.y.values),min(glacier_mask.y.values)))
    glacier_mask = glacier_mask.sel(x=slice( min(xr_dem.x.values), max(xr_dem.x.values) ), y=slice(max(xr_dem.y.values),min(xr_dem.y.values)))

    # flatten dem values into single list, removing 0s
    dem_values = np.ravel(xr_dem.values)
    dem_values = dem_values[ (dem_values!=0) & ~np.isnan(dem_values)]
    dem_length = len(dem_values)
    
    # get the observed aar, ela
    obs_ela = single_obs['ela'].values[0]
    obs_aar = single_obs['aar_infill'].values[0]
    
    if obs_aar>1: obs_aar=1
    
    # define function that will calculate the expected AAR of a given ELA
    def get_hyps_aar(ela_input):
        return round( np.nansum( (dem_values>=ela_input) )/dem_length, 4)
    
    # calculate hypsometric aar, ela
    hyps_ela = np.nanpercentile(dem_values, (1-obs_aar)*100)
    hyps_aar = get_hyps_aar(obs_ela)
    
    # calculate hypsometric AAR if we raise ela by xyz meters
    hyps_aar_20 = get_hyps_aar(obs_ela+20)
    hyps_aar_40 = get_hyps_aar(obs_ela+40)
    hyps_aar_60 = get_hyps_aar(obs_ela+60)
    hyps_aar_80 = get_hyps_aar(obs_ela+80)
    hyps_aar_100 = get_hyps_aar(obs_ela+100)
    hyps_aar_120 = get_hyps_aar(obs_ela+120)
    hyps_aar_140 = get_hyps_aar(obs_ela+140)
    hyps_aar_160 = get_hyps_aar(obs_ela+160)
    hyps_aar_180 = get_hyps_aar(obs_ela+180)
    hyps_aar_200 = get_hyps_aar(obs_ela+200)
    hyps_aar_220 = get_hyps_aar(obs_ela+220)
    hyps_aar_240 = get_hyps_aar(obs_ela+240)
    hyps_aar_260 = get_hyps_aar(obs_ela+260)
    hyps_aar_280 = get_hyps_aar(obs_ela+280)
    hyps_aar_300 = get_hyps_aar(obs_ela+300)
    hyps_aar_320 = get_hyps_aar(obs_ela+320)
    hyps_aar_340 = get_hyps_aar(obs_ela+340)
    hyps_aar_360 = get_hyps_aar(obs_ela+360)
    hyps_aar_380 = get_hyps_aar(obs_ela+380)
    hyps_aar_400 = get_hyps_aar(obs_ela+400)
    hyps_aar_420 = get_hyps_aar(obs_ela+420)
    hyps_aar_440 = get_hyps_aar(obs_ela+440)
    hyps_aar_460 = get_hyps_aar(obs_ela+460)
    hyps_aar_480 = get_hyps_aar(obs_ela+480)
    hyps_aar_500 = get_hyps_aar(obs_ela+500)
    
    # create dictionary to save
    out_dic = {"RGIId":rgiid, "obs_ela":obs_ela, "obs_aar":obs_aar,
               "hyps_ela":hyps_ela, "hyps_aar":hyps_aar,
               "hyps_aar_20":hyps_aar_20, "hyps_aar_40":hyps_aar_40, "hyps_aar_60":hyps_aar_60, "hyps_aar_80":hyps_aar_80, "hyps_aar_100":hyps_aar_100,
               "hyps_aar_120":hyps_aar_120, "hyps_aar_140":hyps_aar_140, "hyps_aar_160":hyps_aar_160, "hyps_aar_180":hyps_aar_180, "hyps_aar_200":hyps_aar_200,
               "hyps_aar_220":hyps_aar_220, "hyps_aar_240":hyps_aar_240, "hyps_aar_260":hyps_aar_260, "hyps_aar_280":hyps_aar_280, "hyps_aar_300":hyps_aar_300,
               "hyps_aar_320":hyps_aar_320, "hyps_aar_340":hyps_aar_340, "hyps_aar_360":hyps_aar_360, "hyps_aar_380":hyps_aar_380, "hyps_aar_400":hyps_aar_400,
               "hyps_aar_420":hyps_aar_420, "hyps_aar_440":hyps_aar_440, "hyps_aar_460":hyps_aar_460, "hyps_aar_480":hyps_aar_480, "hyps_aar_500":hyps_aar_500}
  
    # save to list
    outputs.append(out_dic)

print('Done!')

# format to df, save
out_df = pd.DataFrame(outputs)
out_path = os.path.join(folder_save, 'Average AAs', 'hyspometric_stats.csv')
out_df.to_csv(out_path, index=False)

Starting 1 of 3002: RGI60-01.20889  2.0 km2
Starting 2 of 3002: RGI60-01.11632  2.0 km2
Starting 3 of 3002: RGI60-01.08203  2.002 km2
Starting 4 of 3002: RGI60-01.25183  2.002 km2
Starting 5 of 3002: RGI60-01.11595  2.003 km2
Starting 6 of 3002: RGI60-01.03614  2.004 km2
Starting 7 of 3002: RGI60-01.05487  2.004 km2
Starting 8 of 3002: RGI60-01.09047  2.004 km2
Starting 9 of 3002: RGI60-01.03088  2.005 km2
Starting 10 of 3002: RGI60-01.17509  2.005 km2
Starting 11 of 3002: RGI60-01.10867  2.005 km2
Starting 12 of 3002: RGI60-01.02453  2.006 km2
Starting 13 of 3002: RGI60-01.12977  2.006 km2
Starting 14 of 3002: RGI60-01.08199  2.006 km2
Starting 15 of 3002: RGI60-01.19457  2.007 km2
Starting 16 of 3002: RGI60-01.05254  2.009 km2
Starting 17 of 3002: RGI60-01.02906  2.01 km2
Starting 18 of 3002: RGI60-01.19826  2.012 km2
Starting 19 of 3002: RGI60-01.00315  2.014 km2
Starting 20 of 3002: RGI60-01.00564  2.015 km2
Starting 21 of 3002: RGI60-01.02527  2.016 km2
Starting 22 of 3002: RGI60-

Starting 176 of 3002: RGI60-01.00077  2.13 km2
Starting 177 of 3002: RGI60-01.13830  2.13 km2
Starting 178 of 3002: RGI60-01.00559  2.13 km2
Starting 179 of 3002: RGI60-01.03179  2.132 km2
Starting 180 of 3002: RGI60-01.20866  2.133 km2
Starting 181 of 3002: RGI60-01.10856  2.133 km2
Starting 182 of 3002: RGI60-01.00670  2.134 km2
Starting 183 of 3002: RGI60-01.05455  2.135 km2
Starting 184 of 3002: RGI60-01.09107  2.137 km2
Starting 185 of 3002: RGI60-01.01671  2.138 km2
Starting 186 of 3002: RGI60-01.01786  2.138 km2
Starting 187 of 3002: RGI60-01.06230  2.138 km2
Starting 188 of 3002: RGI60-01.24747  2.139 km2
Starting 189 of 3002: RGI60-01.22792  2.139 km2
Starting 190 of 3002: RGI60-01.23682  2.139 km2
Starting 191 of 3002: RGI60-01.07055  2.14 km2
Starting 192 of 3002: RGI60-01.23254  2.14 km2
Starting 193 of 3002: RGI60-01.04994  2.141 km2
Starting 194 of 3002: RGI60-01.23669  2.142 km2
Starting 195 of 3002: RGI60-01.18960  2.143 km2
Starting 196 of 3002: RGI60-01.13590  2.144 k

Starting 348 of 3002: RGI60-01.01278  2.275 km2
Starting 349 of 3002: RGI60-01.21276  2.275 km2
Starting 350 of 3002: RGI60-01.20198  2.275 km2
Starting 351 of 3002: RGI60-01.07046  2.276 km2
Starting 352 of 3002: RGI60-01.15491  2.277 km2
Starting 353 of 3002: RGI60-01.19385  2.278 km2
Starting 354 of 3002: RGI60-01.20508  2.278 km2
Starting 355 of 3002: RGI60-01.03688  2.278 km2
Starting 356 of 3002: RGI60-01.04161  2.278 km2
Starting 357 of 3002: RGI60-01.05049  2.279 km2
Starting 358 of 3002: RGI60-01.15254  2.279 km2
Starting 359 of 3002: RGI60-01.13917  2.28 km2
Starting 360 of 3002: RGI60-01.16288  2.281 km2
Starting 361 of 3002: RGI60-01.16834  2.281 km2
Starting 362 of 3002: RGI60-01.08681  2.282 km2
Starting 363 of 3002: RGI60-01.01721  2.284 km2
Starting 364 of 3002: RGI60-01.08080  2.284 km2
Starting 365 of 3002: RGI60-01.01331  2.285 km2
Starting 366 of 3002: RGI60-01.19568  2.286 km2
Starting 367 of 3002: RGI60-01.20974  2.286 km2
Starting 368 of 3002: RGI60-01.08472  2.2

Starting 520 of 3002: RGI60-01.03145  2.433 km2
Starting 521 of 3002: RGI60-01.19053  2.436 km2
Starting 522 of 3002: RGI60-01.18932  2.436 km2
Starting 523 of 3002: RGI60-01.20799  2.436 km2
Starting 524 of 3002: RGI60-01.11882  2.439 km2
Starting 525 of 3002: RGI60-01.06210  2.439 km2
Starting 526 of 3002: RGI60-01.15763  2.44 km2
Starting 527 of 3002: RGI60-01.12424  2.44 km2
Starting 528 of 3002: RGI60-01.25200  2.443 km2
Starting 529 of 3002: RGI60-01.01276  2.444 km2
Starting 530 of 3002: RGI60-01.06816  2.446 km2
Starting 531 of 3002: RGI60-01.19656  2.449 km2
Starting 532 of 3002: RGI60-01.12083  2.45 km2
Starting 533 of 3002: RGI60-01.09450  2.451 km2
Starting 534 of 3002: RGI60-01.05250  2.451 km2
Starting 535 of 3002: RGI60-01.03385  2.453 km2
Starting 536 of 3002: RGI60-01.19655  2.454 km2
Starting 537 of 3002: RGI60-01.26463  2.455 km2
Starting 538 of 3002: RGI60-01.08643  2.455 km2
Starting 539 of 3002: RGI60-01.03194  2.46 km2
Starting 540 of 3002: RGI60-01.08425  2.46 k

Starting 692 of 3002: RGI60-01.26315  2.63 km2
Starting 693 of 3002: RGI60-01.08200  2.633 km2
Starting 694 of 3002: RGI60-01.13841  2.633 km2
Starting 695 of 3002: RGI60-01.08410  2.634 km2
Starting 696 of 3002: RGI60-01.08374  2.635 km2
Starting 697 of 3002: RGI60-01.03784  2.636 km2
Starting 698 of 3002: RGI60-01.04055  2.638 km2
Starting 699 of 3002: RGI60-01.03848  2.641 km2
Starting 700 of 3002: RGI60-01.06113  2.644 km2
Starting 701 of 3002: RGI60-01.10925  2.647 km2
Starting 702 of 3002: RGI60-01.20853  2.649 km2
Starting 703 of 3002: RGI60-01.01773  2.65 km2
Starting 704 of 3002: RGI60-01.00901  2.654 km2
Starting 705 of 3002: RGI60-01.08062  2.655 km2
Starting 706 of 3002: RGI60-01.03384  2.656 km2
Starting 707 of 3002: RGI60-01.17905  2.656 km2
Starting 708 of 3002: RGI60-01.08076  2.656 km2
Starting 709 of 3002: RGI60-01.23757  2.656 km2
Starting 710 of 3002: RGI60-01.12281  2.657 km2
Starting 711 of 3002: RGI60-01.01140  2.658 km2
Starting 712 of 3002: RGI60-01.13521  2.66

...

Starting 2216 of 3002: RGI60-01.14332  9.386 km2
Starting 2217 of 3002: RGI60-01.17902  9.404 km2
Starting 2218 of 3002: RGI60-01.22222  9.496 km2
Starting 2219 of 3002: RGI60-01.20926  9.518 km2
Starting 2220 of 3002: RGI60-01.01104  9.528 km2
Starting 2221 of 3002: RGI60-01.22715  9.535 km2
Starting 2222 of 3002: RGI60-01.16847  9.581 km2
Starting 2223 of 3002: RGI60-01.19454  9.583 km2
Starting 2224 of 3002: RGI60-01.16867  9.593 km2
Starting 2225 of 3002: RGI60-01.00242  9.635 km2
Starting 2226 of 3002: RGI60-01.01607  9.649 km2
Starting 2227 of 3002: RGI60-01.04979  9.66 km2
Starting 2228 of 3002: RGI60-01.09344  9.68 km2
Starting 2229 of 3002: RGI60-01.19616  9.697 km2
Starting 2230 of 3002: RGI60-01.15950  9.71 km2
Starting 2231 of 3002: RGI60-01.04968  9.728 km2
Starting 2232 of 3002: RGI60-01.13622  9.738 km2
Starting 2233 of 3002: RGI60-01.17480  9.757 km2
Starting 2234 of 3002: RGI60-01.05331  9.771 km2
Starting 2235 of 3002: RGI60-01.11307  9.772 km2
Starting 2236 of 3002: 

Starting 2382 of 3002: RGI60-01.15689  13.448 km2
Starting 2383 of 3002: RGI60-01.10928  13.507 km2
Starting 2384 of 3002: RGI60-01.09641  13.525 km2
Starting 2385 of 3002: RGI60-01.08434  13.527 km2
Starting 2386 of 3002: RGI60-01.09462  13.553 km2
Starting 2387 of 3002: RGI60-01.26719  13.612 km2
Starting 2388 of 3002: RGI60-01.08336  13.637 km2
Starting 2389 of 3002: RGI60-01.01486  13.651 km2
Starting 2390 of 3002: RGI60-01.20967  13.673 km2
Starting 2391 of 3002: RGI60-01.04586  13.706 km2
Starting 2392 of 3002: RGI60-01.20742  13.754 km2
Starting 2393 of 3002: RGI60-01.09656  13.791 km2
Starting 2394 of 3002: RGI60-01.00016  13.795 km2
Starting 2395 of 3002: RGI60-01.19724  13.86 km2
Starting 2396 of 3002: RGI60-01.06051  13.877 km2
Starting 2397 of 3002: RGI60-01.09124  13.897 km2
Starting 2398 of 3002: RGI60-01.05995  13.946 km2
Starting 2399 of 3002: RGI60-01.01808  13.959 km2
Starting 2400 of 3002: RGI60-01.23638  13.962 km2
Starting 2401 of 3002: RGI60-01.19664  13.995 km2
S

Starting 2547 of 3002: RGI60-01.24715  20.882 km2
Starting 2548 of 3002: RGI60-01.01282  20.978 km2
Starting 2549 of 3002: RGI60-01.15644  21.055 km2
Starting 2550 of 3002: RGI60-01.03514  21.071 km2
Starting 2551 of 3002: RGI60-01.20840  21.093 km2
Starting 2552 of 3002: RGI60-01.09761  21.314 km2
Starting 2553 of 3002: RGI60-01.20297  21.359 km2
Starting 2554 of 3002: RGI60-01.22230  21.394 km2
Starting 2555 of 3002: RGI60-01.20868  21.56 km2
Starting 2556 of 3002: RGI60-01.16789  21.594 km2
Starting 2557 of 3002: RGI60-01.16264  21.878 km2
Starting 2558 of 3002: RGI60-01.15338  21.891 km2
Starting 2559 of 3002: RGI60-01.15547  21.986 km2
Starting 2560 of 3002: RGI60-01.19552  22.057 km2
Starting 2561 of 3002: RGI60-01.03677  22.188 km2
Starting 2562 of 3002: RGI60-01.14847  22.412 km2
Starting 2563 of 3002: RGI60-01.21000  22.455 km2
Starting 2564 of 3002: RGI60-01.16316  22.496 km2
Starting 2565 of 3002: RGI60-01.01482  22.598 km2
Starting 2566 of 3002: RGI60-01.03502  22.638 km2
S

Starting 2712 of 3002: RGI60-01.12188  39.789 km2
Starting 2713 of 3002: RGI60-01.18059  39.858 km2
Starting 2714 of 3002: RGI60-01.15731  40.009 km2
Starting 2715 of 3002: RGI60-01.08296  40.202 km2
Starting 2716 of 3002: RGI60-01.20740  40.235 km2
Starting 2717 of 3002: RGI60-01.01306  40.313 km2
Starting 2718 of 3002: RGI60-01.20188  40.401 km2
Starting 2719 of 3002: RGI60-01.02696  40.463 km2
Starting 2720 of 3002: RGI60-01.01519  40.504 km2
Starting 2721 of 3002: RGI60-01.21515  40.834 km2
Starting 2722 of 3002: RGI60-01.10979  40.893 km2
Starting 2723 of 3002: RGI60-01.04401  41.026 km2
Starting 2724 of 3002: RGI60-01.21372  41.056 km2
Starting 2725 of 3002: RGI60-01.17988  41.538 km2
Starting 2726 of 3002: RGI60-01.17569  41.542 km2
Starting 2727 of 3002: RGI60-01.16110  41.61 km2
Starting 2728 of 3002: RGI60-01.09798  41.785 km2
Starting 2729 of 3002: RGI60-01.25542  42.575 km2
Starting 2730 of 3002: RGI60-01.14331  42.592 km2
Starting 2731 of 3002: RGI60-01.22187  42.67 km2
St

Starting 2876 of 3002: RGI60-01.20968  113.425 km2
Starting 2877 of 3002: RGI60-01.21001  114.988 km2
Starting 2878 of 3002: RGI60-01.09211  115.1 km2
Starting 2879 of 3002: RGI60-01.06025  115.119 km2
Starting 2880 of 3002: RGI60-01.21081  115.277 km2
Starting 2881 of 3002: RGI60-01.13601  118.851 km2
Starting 2882 of 3002: RGI60-01.01733  119.101 km2
Starting 2883 of 3002: RGI60-01.20791  120.353 km2
Starting 2884 of 3002: RGI60-01.13638  121.133 km2
Starting 2885 of 3002: RGI60-01.15772  121.567 km2
Starting 2886 of 3002: RGI60-01.10607  121.571 km2
Starting 2887 of 3002: RGI60-01.09471  122.352 km2
Starting 2888 of 3002: RGI60-01.19165  125.008 km2
Starting 2889 of 3002: RGI60-01.14186  129.109 km2
Starting 2890 of 3002: RGI60-01.10836  129.55 km2
Starting 2891 of 3002: RGI60-01.23635  129.755 km2
Starting 2892 of 3002: RGI60-01.00566  131.5 km2
Starting 2893 of 3002: RGI60-01.27105  131.574 km2
Starting 2894 of 3002: RGI60-01.19825  132.446 km2
Starting 2895 of 3002: RGI60-01.0374

In [9]:
print(f"Given observed ELA of: {obs_ela}, we expect AAR of: {hyps_aar}")
print(f"Given observed AAR of: {obs_aar}, we expect ELA of: {hyps_ela}")
print(f"If ELA rises by 100 meters, the hypsometric AAR will be: {hyps_aar_100}")
print(f"If ELA rises by 200 meters, the hypsometric AAR will be: {hyps_aar_200}")
print(f"If ELA rises by 500 meters, the hypsometric AAR will be: {hyps_aar_500}")

Given observed ELA of: 1450, we expect AAR of: 0.4981
Given observed AAR of: 0.4941, we expect ELA of: 1484.0
If ELA rises by 100 meters, the hypsometric AAR will be: 0.48
If ELA rises by 200 meters, the hypsometric AAR will be: 0.4439
If ELA rises by 500 meters, the hypsometric AAR will be: 0.1962


In [6]:
out_df

,RGIId,obs_ela,obs_aar,hyps_ela,hyps_aar,hyps_aar_20,hyps_aar_40,hyps_aar_60,hyps_aar_80,hyps_aar_100,...,hyps_aar_320,hyps_aar_340,hyps_aar_360,hyps_aar_380,hyps_aar_400,hyps_aar_420,hyps_aar_440,hyps_aar_460,hyps_aar_480,hyps_aar_500
0,RGI60-01.20889,1400,0.5658,1432.0,0.6919,0.6083,0.5400,0.4647,0.3790,0.3101,...,0.0133,0.0096,0.0077,0.0046,0.0032,0.0014,0.0000,0.0000,0.0000,0.0000
1,RGI60-01.11632,1710,0.0374,1667.0,0.0095,0.0029,0.0001,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,RGI60-01.08203,1170,0.2052,1074.0,0.0109,0.0008,0.0001,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,RGI60-01.25183,2440,0.3425,2420.0,0.2573,0.1931,0.1510,0.1271,0.1073,0.0935,...,0.0191,0.0148,0.0099,0.0065,0.0033,0.0003,0.0000,0.0000,0.0000,0.0000
4,RGI60-01.11595,1430,0.3683,1494.0,0.4941,0.4554,0.4175,0.3773,0.3360,0.2982,...,0.0417,0.0344,0.0284,0.0190,0.0120,0.0065,0.0030,0.0007,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2997,RGI60-01.16201,2000,0.5914,2150.0,0.6871,0.6716,0.6561,0.6428,0.6314,0.6206,...,0.4638,0.4473,0.4314,0.4149,0.3986,0.3809,0.3641,0.3468,0.3282,0.3103
2998,RGI60-01.17566,2130,0.4927,2213.0,0.5626,0.5431,0.5260,0.5110,0.4955,0.4801,...,0.2963,0.2831,0.2710,0.2589,0.2474,0.2361,0.2254,0.2157,0.2071,0.1992
2999,RGI60-01.14443,1080,0.8673,1312.0,0.9543,0.9493,0.9396,0.9319,0.9255,0.9171,...,0.8078,0.7937,0.7814,0.7697,0.7588,0.7483,0.7343,0.7230,0.7131,0.7019
3000,RGI60-01.13635,1400,0.4987,1502.0,0.5445,0.5361,0.5288,0.5200,0.5119,0.5004,...,0.4224,0.4141,0.4032,0.3917,0.3833,0.3747,0.3669,0.3585,0.3499,0.3392


In [7]:
single_obs

,RGIId,ela,aar,off_glacier,total_area,total_observed,total_snow,total_snow_infill,aar_infill
3001,RGI60-01.13696,1450,0.4678,0,3376.8262,3162.1795,1479.2369,1668.4994,0.4941


In [8]:
df_obs

,RGIId,ela,aar,off_glacier,total_area,total_observed,total_snow,total_snow_infill,aar_infill
0,RGI60-01.20889,1400,0.6835,0,2.0445,1.1689,0.7990,1.1567,0.5658
1,RGI60-01.11632,1710,0.0411,1,2.0436,1.8608,0.0764,0.0764,0.0374
2,RGI60-01.08203,1170,0.2218,1,2.0598,1.9057,0.4226,0.4226,0.2052
3,RGI60-01.25183,2440,0.2939,0,2.0824,1.8965,0.5574,0.7133,0.3425
4,RGI60-01.11595,1430,0.2666,0,2.0488,1.3949,0.3719,0.7546,0.3683
...,...,...,...,...,...,...,...,...,...
2997,RGI60-01.16201,2000,0.5523,0,1062.3297,961.2816,530.9023,628.2290,0.5914
2998,RGI60-01.17566,2130,0.4140,0,1187.7931,1017.7733,421.3875,585.1992,0.4927
2999,RGI60-01.14443,1080,0.8544,0,2854.2166,2528.4904,2160.4047,2475.3661,0.8673
3000,RGI60-01.13635,1400,0.4722,0,3040.1670,2849.1098,1345.4882,1516.2110,0.4987
